In [67]:
import time
import gc
import gudhi.representations
import numpy as np
import pandas as pd
import gudhi as gd 
import subprocess as sp
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from persim.persistent_entropy import *
from utils import *
pca = PCA(n_components=2)

In [117]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

housing = fetch_california_housing()

X_all = pd.DataFrame(housing.data, columns = housing.feature_names)
y_all = housing['target']

scaler = StandardScaler()
scaler.fit(X_all)
X_scaled = pd.DataFrame(scaler.transform(X_all), index= X_all.index, columns= X_all.columns)

pca_result = pca.fit_transform(X_scaled)
pca_result = pd.DataFrame(pca_result)
X_scaled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.344766,0.982143,0.628559,-0.153758,-0.974429,-0.049597,1.052548,-1.327835
1,2.332238,-0.607019,0.327041,-0.263336,0.861439,-0.092512,1.043185,-1.322844
2,1.782699,1.856182,1.155620,-0.049016,-0.820777,-0.025843,1.038503,-1.332827
3,0.932968,1.856182,0.156966,-0.049833,-0.766028,-0.050329,1.038503,-1.337818
4,-0.012881,1.856182,0.344711,-0.032906,-0.759847,-0.085616,1.038503,-1.337818


In [118]:
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, fcluster

Z = linkage(pca_result, method='ward')
hierarchical_labels = fcluster(Z, t=5, criterion='maxclust')

print("Cluster labels:", hierarchical_labels)

Cluster labels: [3 3 3 ... 3 3 3]


In [119]:
labels_series = pd.Series(hierarchical_labels)

labels_series.value_counts()

3    9461
2    8548
1    2512
5     117
4       2
dtype: int64

In [145]:
pca_result = pd.DataFrame(pca_result)
pca_result['label'] = hierarchical_labels
pca_result['y_all'] = y_all

group_0 = pca_result[pca_result['label'] == 1]
group_1 = pca_result[pca_result['label'] == 2]
group_2 = pca_result[pca_result['label'] == 3]
group_3 = pca_result[pca_result['label'] == 4]
group_4 = pca_result[pca_result['label'] == 5]

group_0 = group_0.drop(columns=['label'])
group_1 = group_1.drop(columns=['label'])
group_2 = group_2.drop(columns=['label'])
group_3 = group_3.drop(columns=['label'])
group_4 = group_4.drop(columns=['label'])

group_0 = group_0.reset_index(drop=True)
group_1 = group_1.reset_index(drop=True)
group_2 = group_2.reset_index(drop=True)
group_3 = group_3.reset_index(drop=True)
group_4 = group_4.reset_index(drop=True)

print(group_0.shape)
print(group_1.shape)
print(group_2.shape)
print(group_3.shape)
print(group_4.shape)

(2512, 3)
(8548, 3)
(9461, 3)
(2, 3)
(117, 3)


In [146]:
# lst = [group_0, group_1, group_2, group_3, group_4]
lst = [group_4]
rmses = []
rsquares = []

for group in lst:
    
    data_list = Data_list(group.drop(columns = ['y_all']))
    results = Parrallel_compute(data_list, typeComplex='Alpha')

    data = X_scaled[pca_result['label'] == 5].copy()
    data['topo_feature'] = [x.tolist()[0][0] if isinstance(x, np.ndarray) else x for x in results]
    data['topo_feature'] = scaler.fit_transform(data[['topo_feature']])

    X_train, X_test, y_train, y_test = train_test_split(
        data, group['y_all'], test_size=0.5, random_state=42)
    
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train = pd.Series(y_train).reset_index(drop=True)
    y_test = pd.Series(y_test).reset_index(drop=True)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    predictions_lr = lr.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
    r2 = r2_score(y_test, predictions_lr)

    rmses.append(rmse)
    rsquares.append(r2)

平行運算時間:  0.13063406944274902


In [147]:
print(rmses)
print(rsquares)

[1.1590494740486847]
[-1.7406897764527551]
